# Import Libraries

In [ ]:
import numpy as np
import pandas as pd
import plotly.graph_objs as go
import plotly.offline as pyo
from jupyter_dash import JupyterDash
import dash
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
import numpy.random as random
random.seed(101)

# Import Data

In [ ]:
df = pd.read_csv("./data/mpg.csv")
df = df[df["horsepower"].str.isnumeric()]
df["horsepower"] = df["horsepower"].astype(float)
jitter = random.randint(low=-4, high=5, size=len(df))*0.1
df["year"] = 1900 + df["model_year"] + jitter
df.head()

# Create App

In [ ]:
app = JupyterDash(__name__)

# Modify App Layout

Comment out only one of the cells in this section to run the Application with the selected App layout.

Highlight the contents of the cell and use the shortcut key ```Ctrl``` + ```/``` to comment/uncomment all.

In [ ]:
# data1 = [go.Scatter(x=df["year"], y=df["mpg"], text=df["name"], hoverinfo= "text+y+x", mode="markers")]
# layout1 = go.Layout(title="MPG Data", xaxis=dict(title= "Model Year"), yaxis=dict(title="MPG"))
# fig1 = go.Figure(data=data1, layout=layout1)

# app.layout = html.Div([dcc.Graph(id="mpg-scatter", figure=fig1)
#                       ]
#                      )

In [ ]:
# data1 = [go.Scatter(x=df["year"], y=df["mpg"], text=df["name"], hoverinfo= "text+y+x", mode="markers")]
# layout1 = go.Layout(title="MPG Data", xaxis=dict(title= "Model Year"), yaxis=dict(title="MPG"), hovermode="closest")
# fig1 = go.Figure(data=data1, layout=layout1)

# data2 = [go.Scatter(x=[0, 1], y=[0, 1], mode="lines")]
# layout2 = go.Layout(title="Acceleration",
#                     xaxis=dict(visible=False),
#                     yaxis=dict(visible=False, range=[0,60/df["acceleration"].min()]), 
#                     margin=dict(l=0), height=300)
# fig2 = go.Figure(data=data2, layout=layout2)

# app.layout = html.Div([html.Div(dcc.Graph(id="mpg-scatter", figure=fig1), style={"width": "50%", "display": "inline-block"}),
#                        html.Div(dcc.Graph(id="mpg-line", figure=fig2),  style={"width": "50%", "display": "inline-block"})
#                       ]
#                      )

# @app.callback(Output("mpg-line", "figure"),
#               Input("mpg-scatter", "hoverData"))
# def callback_graph(hoverData):
#     if hoverData!=None:
#         v_index = hoverData["points"][0]["pointIndex"]
#         data2 = [go.Scatter(x=[0, 1], y=[0, 60/df.iloc[v_index]["acceleration"]], mode="lines")]
#         layout2 = go.Layout(title=df.iloc[v_index]["name"], 
#                             xaxis=dict(visible=False), 
#                             yaxis=dict(visible=False, range=[0,60/df["acceleration"].min()]), 
#                             margin=dict(l=0), height=300)
#         fig2 = go.Figure(data=data2, layout=layout2)
#         return fig2
#     else:
#         data2 = [go.Scatter(x=[0, 1], y=[0, 1], mode="lines")]
#         layout2 = go.Layout(title="Acceleration",                             
#                             xaxis=dict(visible=False), 
#                             yaxis=dict(visible=False, range=[0,60/df["acceleration"].min()]), 
#                             margin=dict(l=0), height=300)
#         fig2 = go.Figure(data=data2, layout=layout2)
#         return fig2

In [ ]:
data1 = [go.Scatter(x=df["year"], y=df["mpg"], text=df["name"], hoverinfo= "text+y+x", mode="markers")]
layout1 = go.Layout(title="MPG Data", xaxis=dict(title= "Model Year"), yaxis=dict(title="MPG"), hovermode="closest")
fig1 = go.Figure(data=data1, layout=layout1)

data2 = [go.Scatter(x=[0, 1], y=[0, 1], mode="lines")]
layout2 = go.Layout(title="Acceleration",
                    xaxis=dict(visible=False),
                    yaxis=dict(visible=False, range=[0,60/df["acceleration"].min()]), 
                    margin=dict(l=0), height=300)
fig2 = go.Figure(data=data2, layout=layout2)

app.layout = html.Div([html.Div(dcc.Graph(id="mpg-scatter", figure=fig1), style={"width": "50%", "display": "inline-block", 
                                                                                 "vertical-align": "middle"}),
                       html.Div(dcc.Graph(id="mpg-line", figure=fig2),  style={"width": "20%", "height": "50%", "display": "inline-block",
                                                                               "vertical-align": "middle"}),
                       html.Div(dcc.Markdown(id="mpg-stats"), style={"width": "20%", "height": "50%", "display": "inline-block",
                                                                     "vertical-align": "middle"})
                      ]
                     )

@app.callback(Output("mpg-line", "figure"),
              Input("mpg-scatter", "hoverData"))
def callback_graph(hoverData):
    if hoverData!=None:
        v_index = hoverData["points"][0]["pointIndex"]
        data2 = [go.Scatter(x=[0, 1], y=[0, 60/df.iloc[v_index]["acceleration"]], mode="lines",
                           line=dict(width=3*df.iloc[v_index]["cylinders"]))]
        layout2 = go.Layout(title=df.iloc[v_index]["name"], 
                            xaxis=dict(visible=False), 
                            yaxis=dict(visible=False, range=[0,60/df["acceleration"].min()]), 
                            margin=dict(l=0), height=300)
        fig2 = go.Figure(data=data2, layout=layout2)
        return fig2
    else:
        data2 = [go.Scatter(x=[0, 1], y=[0, 1], mode="lines")]
        layout2 = go.Layout(title="Acceleration",                             
                            xaxis=dict(visible=False), 
                            yaxis=dict(visible=False, range=[0,60/df["acceleration"].min()]), 
                            margin=dict(l=0), height=300)
        fig2 = go.Figure(data=data2, layout=layout2)
        return fig2
    
@app.callback(Output("mpg-stats", "children"),
              Input("mpg-scatter", "hoverData"))
def callback_stats(hoverData):
    if hoverData!=None:
        v_index = hoverData["points"][0]["pointIndex"]
        stats = f"""
                {df.iloc[v_index]["cylinders"]} cylinders
                {df.iloc[v_index]["displacement"]} cc displacement
                0 to 60 mph in {df.iloc[v_index]["acceleration"]} seconds
        """
        return stats
    else:
        return "No selection"
    

# Run App 

## In Cell

In [ ]:
app.run_server(mode="inline", height=650, host='127.0.0.1', port='8050')

Go to Restart Kernel and Clear All Outputs to End the Server.

## In a New Tab (Inside JupyterLab)

In [ ]:
# app.run_server(mode="jupyterlab")

Go to Restart Kernel and Clear All Outputs to End the Server.

## In a New Tab

In [ ]:
# app.run_server()

Go to Restart Kernel and Clear All Outputs to End the Server.